In [6]:
!module load cuda/11.2.0

In [1]:
# Import functions
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from collections import Counter 

2025-06-25 21:10:19.188557: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-25 21:10:19.188669: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-06-25 21:10:19.350748: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-25 21:10:23.473891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-06-25 21:10:23.475042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
# Genus/species labels
taxon_labels = '/scicomp/home-pure/rqu4/PROJECTS/GaTech/FCGR_classifier/data_final/enterobacteriaceae/metadata/metadata_tax_assignments.csv'

df_tax = pd.read_csv(taxon_labels, sep=',')

df_tax['genus'] = df_tax['organism_organism_name'].str.split('_').str[0]
df_tax['species'] = df_tax['organism_organism_name'].str.split('_').str[1]
df_tax['species'] = df_tax['species'].replace("sp.", np.nan)
df_tax.head()


,accession,organism_organism_name,genus,species
0,GCF_001461805.1,Enterobacter_lignolyticus,Enterobacter,lignolyticus
1,GCA_001461805.1,Enterobacter_lignolyticus,Enterobacter,lignolyticus
2,GCF_000164865.1,Enterobacter_lignolyticus_SCF1,Enterobacter,lignolyticus
3,GCA_000164865.1,Enterobacter_lignolyticus_SCF1,Enterobacter,lignolyticus
4,GCF_001856865.2,Kluyvera_intestini,Kluyvera,intestini


In [3]:
# FCGR_arrays
FCGR_array_dir = '/scicomp/home-pure/rqu4/PROJECTS/GaTech/FCGR_classifier/data_final/enterobacteriaceae/data/FCGR_arrays'

contents = os.listdir(FCGR_array_dir)

print(contents[:5])

['GCF_000005845.2_ASM584v2_genomic_k5_k5.npy', 'GCF_000006925.2_ASM692v2_genomic_k5_k5.npy', 'GCF_000008105.1_ASM810v1_genomic_k5_k5.npy', 'GCF_000007445.1_ASM744v1_genomic_k5_k5.npy', 'GCF_000007405.1_ASM740v1_genomic_k5_k5.npy']


In [4]:
# # Example FCGR array

e_FCGR = FCGR_array_dir + '/' + 'GCF_000005845.2_ASM584v2_genomic_k5_k5.npy'

array = np.load(e_FCGR)

print(array.shape)
print(array)

(32, 32)
[[ 2975.  3687.  6749. ... 10000. 10000. 10000.]
 [ 5661.     0.  3735. ... 10000. 10000. 10000.]
 [10000.  6415.  3771. ... 10000.  4935. 10000.]
 ...
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.  3669.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]]


In [6]:
# --- Configuration ---
FCGR_array_dir = '/scicomp/home-pure/rqu4/PROJECTS/GaTech/FCGR_classifier/data_final/enterobacteriaceae/data/FCGR_arrays' # Your actual path
MIN_REPRESENTATIVES = 12
SPECIES_COLUMN = 'species' # Ensure this matches your actual species column name in df_tax
ACCESSION_COLUMN = 'accession' # Ensure this matches your actual accession column name in df_tax

# --- Assumed: df_tax is already loaded here ---
# Example: df_tax = pd.read_csv('your_taxonomic_data.csv')

print(f"Original df_tax shape: {df_tax.shape}")
print(f"Original unique species count: {df_tax[SPECIES_COLUMN].nunique()}")
print(f"Original species value counts (top 10):\n{df_tax[SPECIES_COLUMN].value_counts().head(10)}")

# --- Filter df_tax for species with at least MIN_REPRESENTATIVES ---
species_counts = df_tax[SPECIES_COLUMN].value_counts()
species_to_keep = species_counts[species_counts >= MIN_REPRESENTATIVES].index.tolist()
df_tax_filtered = df_tax[df_tax[SPECIES_COLUMN].isin(species_to_keep)].copy() # Use .copy() to avoid SettingWithCopyWarning

print(f"\nFiltered df_tax_filtered shape (min {MIN_REPRESENTATIVES} reps per species): {df_tax_filtered.shape}")
print(f"Unique species count in df_tax_filtered: {df_tax_filtered[SPECIES_COLUMN].nunique()}")
print(f"Filtered species value counts (top 10):\n{df_tax_filtered[SPECIES_COLUMN].value_counts().head(10)}")

# --- FCGR Data Loading (modified to use df_tax_filtered) ---
fcgr_data = []
species_labels = []

fcgr_files = [f for f in os.listdir(FCGR_array_dir) if f.endswith('.npy')]

processed_count = 0
skipped_count = 0

print("\nStarting FCGR array loading and matching with filtered species...")
# Create a set of accessions and a dictionary for faster lookup
filtered_accessions_set = set(df_tax_filtered[ACCESSION_COLUMN].tolist())
species_lookup = df_tax_filtered.set_index(ACCESSION_COLUMN)[SPECIES_COLUMN].to_dict()

for file_name in fcgr_files:
    # Extract the accession from the filename, e.g., 'GCF_000005845.2'
    file_accession = '_'.join(file_name.split('_')[:2])

    # Check if the accession is in our filtered set
    if file_accession in filtered_accessions_set:
        species = species_lookup.get(file_accession)
        
        # This check is mostly for robustness, should not be None if filtered_accessions_set is used correctly
        if species is None:
            skipped_count += 1
            continue

        array_path = os.path.join(FCGR_array_dir, file_name)

        try:
            fcgr_array = np.load(array_path)
            
            if fcgr_array.shape == (32, 32):
                fcgr_data.append(fcgr_array)
                species_labels.append(species)
                processed_count += 1
            else:
                skipped_count += 1
        except Exception as e:
            print(f"Error loading or processing {file_name}: {e}")
            skipped_count += 1
    else:
        # Accession not in filtered tax data, so skip
        skipped_count += 1

print(f"\nSuccessfully processed {processed_count} FCGR arrays.")
print(f"Skipped {skipped_count} FCGR arrays (either no tax data in filtered set, or wrong shape).")

if not fcgr_data:
    raise ValueError(
        "No FCGR data loaded after processing. "
        "Please check your FCGR_array_dir, df_tax loading, accession matching logic, "
        "and the species filtering threshold. No data matched the criteria."
    )

X = np.array(fcgr_data)
y = np.array(species_labels)

# --- Preprocess Labels and Features ---
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# num_classes = len(label_encoder.classes_) # This might be your issue if used directly in model

y_one_hot = to_categorical(y_encoded, num_classes=len(label_encoder.classes_)) # Use len(label_encoder.classes_) here for original one-hot
X_flattened = X.reshape(X.shape[0], -1)

# Check for problematic classes (only 1 member) before splitting for stratify
class_counts = Counter(y_encoded)
single_member_classes = [cls for cls, count in class_counts.items() if count == 1]

if single_member_classes:
    print("\nWARNING: Classes with only 1 member found after initial filtering. These will be removed for stratification.")
    for cls in single_member_classes:
        original_label = label_encoder.inverse_transform([cls])[0]
        print(f"  - Class {cls} ({original_label})")

    indices_to_keep = []
    for i, cls in enumerate(y_encoded):
        if cls not in single_member_classes:
            indices_to_keep.append(i)

    X_to_split = X_flattened[indices_to_keep]
    y_encoded_to_split = y_encoded[indices_to_keep]
    y_one_hot_to_split = y_one_hot[indices_to_keep]

    # --- THIS IS THE CRITICAL PART ---
    # After filtering, re-encode y_encoded_to_split to get new, contiguous numerical labels
    # and accurately count the NEW number of classes.
    final_label_encoder = LabelEncoder() # Create a NEW LabelEncoder for the filtered data
    y_encoded_to_split_reencoded = final_label_encoder.fit_transform(y_encoded_to_split)
    num_classes_final = len(final_label_encoder.classes_) # This is the correct number of classes
    
    # And re-one-hot encode with the new number of classes
    y_one_hot_to_split = to_categorical(y_encoded_to_split_reencoded, num_classes=num_classes_final)

    # Update the data to be used for splitting
    y_encoded_to_split = y_encoded_to_split_reencoded # Use the re-encoded y for stratify

    print(f"Adjusted data shape for splitting: X={X_to_split.shape}, y={y_one_hot_to_split.shape}")
    print(f"Number of classes after final adjustment: {num_classes_final}") # Use this for model output

else:
    print("\nNo single-member classes found. Proceeding with split.")
    X_to_split = X_flattened
    y_one_hot_to_split = y_one_hot
    y_encoded_to_split = y_encoded
    num_classes_final = num_classes # This was set earlier from original label_encoder.classes_

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_to_split, y_one_hot_to_split, test_size=0.2, random_state=42, stratify=y_encoded_to_split
)

print(f"\nShape of input features for training: {X_train.shape}")
print(f"Shape of target labels for training: {y_train.shape}")
print(f"Number of samples for training: {X_train.shape[0]}")
print(f"Number of samples for testing: {X_test.shape[0]}")
print(f"Final number of unique species classes used: {num_classes_final}")


# --- Build the MLP Model ---
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes_final, activation='softmax') # Use the CORRECTED number of classes here
])

# --- Train the Model ---
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\n--- Model Summary ---")
model.summary()

print("\n--- Training Model ---")
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# --- Evaluate the Model ---
print("\n--- Evaluating Model ---")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# --- Optional: Make predictions on test data ---
print("\n--- Sample Predictions ---")
predictions = model.predict(X_test)
predicted_species_indices = np.argmax(predictions, axis=1)
true_species_indices = np.argmax(y_test, axis=1)

for i in range(min(10, len(X_test))):
    # Map back to original labels using the *original* label_encoder
    predicted_species = label_encoder.inverse_transform([predicted_species_indices[i]])[0]
    true_species = label_encoder.inverse_transform([true_species_indices[i]])[0]
    print(f"Sample {i+1}: True Species: {true_species}, Predicted Species: {predicted_species}")

Original df_tax shape: (869903, 4)
Original unique species count: 170
Original species value counts (top 10):
species
enterica        390912
coli            387273
sonnei           25211
flexneri         21970
hormaechei       14640
freundii          4888
sakazakii         3068
cloacae           2620
asburiae          1444
roggenkampii      1408
Name: count, dtype: int64

Filtered df_tax_filtered shape (min 12 reps per species): (864748, 4)
Unique species count in df_tax_filtered: 72
Filtered species value counts (top 10):
species
enterica        390912
coli            387273
sonnei           25211
flexneri         21970
hormaechei       14640
freundii          4888
sakazakii         3068
cloacae           2620
asburiae          1444
roggenkampii      1408
Name: count, dtype: int64

Starting FCGR array loading and matching with filtered species...

Successfully processed 6406 FCGR arrays.
Skipped 4528 FCGR arrays (either no tax data in filtered set, or wrong shape).

  - Class 37 (muyt

2025-06-25 21:12:47.812877: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-25 21:12:47.814083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-06-25 21:12:47.815028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-06-25 21:12:47.815945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2025-06-25 21:12:47.816788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64


--- Model Summary ---
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 44)                2860      
                                                                 
Total params: 306,412
Trainable params: 306,412
Non-trainable params: 0
_________________________________________________________________

--- Training Model ---
Epoch 1/50
144/144 [==============================] - 2s 8ms/step - loss: 901.0341 - accuracy: 0.5139 - val_loss: 262.6064 - val_accura

In [7]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score

# --- Train/Test Split for XGBoost ---
X_train, X_test, y_train, y_test = train_test_split(
    X_to_split, y_encoded_to_split, test_size=0.2, random_state=42, stratify=y_encoded_to_split
)

print(f"\n[INFO] XGBoost Training Samples: {X_train.shape[0]}, Test Samples: {X_test.shape[0]}")

# --- Train XGBoost Classifier ---
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',         # or 'multi:softprob' if you want probabilities
    num_class=num_classes_final,
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)




[INFO] XGBoost Training Samples: 5112, Test Samples: 1278


/scicomp/home-pure/rqu4/cache/python/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [21:43:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_class=44, num_parallel_tree=None, ...)

In [12]:
from sklearn.metrics import classification_report
from sklearn.utils.multiclass import unique_labels

# Ensure labels are standard Python ints
used_labels = [int(i) for i in unique_labels(y_test, y_pred)]

# Ensure class names are properly extracted as strings
target_names = [str(final_label_encoder.classes_[i]) for i in used_labels]

print(classification_report(
    y_test,
    y_pred,
    labels=used_labels,
    target_names=target_names
))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           5       1.00      1.00      1.00        15
           6       1.00      0.67      0.80         3
           7       0.85      1.00      0.92        11
           8       1.00      0.33      0.50         3
           9       1.00      0.88      0.93         8
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         1
          13       0.00      0.00      0.00         1
          14       1.00      0.73      0.84        11
          15       0.98      1.00      0.99       766
          16       1.00      1.00      1.00         1
          18       1.00      1.00      1.00         1
          19       1.00      0.33      0.50         6
          20       1.00      1.00      1.00         4
          21       1.00      1.00      1.00       232
          23       0.00    

/scicomp/home-pure/rqu4/cache/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scicomp/home-pure/rqu4/cache/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scicomp/home-pure/rqu4/cache/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [13]:
print("\n--- Sample Predictions ---")
for i in range(min(10, len(X_test))):
    pred_label = final_label_encoder.inverse_transform([y_pred[i]])[0]
    true_label = final_label_encoder.inverse_transform([y_test[i]])[0]
    print(f"Sample {i+1}: True Species: {true_label}, Predicted Species: {pred_label}")



--- Sample Predictions ---
Sample 1: True Species: 15, Predicted Species: 15
Sample 2: True Species: 15, Predicted Species: 15
Sample 3: True Species: 15, Predicted Species: 15
Sample 4: True Species: 21, Predicted Species: 21
Sample 5: True Species: 15, Predicted Species: 15
Sample 6: True Species: 15, Predicted Species: 15
Sample 7: True Species: 15, Predicted Species: 15
Sample 8: True Species: 21, Predicted Species: 21
Sample 9: True Species: 15, Predicted Species: 15
Sample 10: True Species: 15, Predicted Species: 15


In [39]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.utils.multiclass import unique_labels

# --- Step 0: Filter classes with at least 12 samples ---

# Assume:
# - X_to_split: your full feature matrix, shape (N_samples, N_features)
# - y_encoded_to_split: your original encoded labels (int or str)
# Note: y_encoded_to_split can be original labels, but encoding below will handle mapping

# Convert to numpy arrays if not already
X_to_split = np.array(X_to_split)
y_encoded_to_split = np.array(y_encoded_to_split)

# Count samples per class
(unique, counts) = np.unique(y_encoded_to_split, return_counts=True)
class_counts = dict(zip(unique, counts))

# Select classes with >=12 samples
classes_to_keep = [cls for cls, count in class_counts.items() if count >= 50]

# Filter data
indices_to_keep = [i for i, label in enumerate(y_encoded_to_split) if label in classes_to_keep]
X_filtered = X_to_split[indices_to_keep]
y_filtered = y_encoded_to_split[indices_to_keep]

print(f"[INFO] Classes before filtering: {len(class_counts)}")
print(f"[INFO] Classes after filtering (>=12 samples): {len(classes_to_keep)}")
print(f"[INFO] Samples after filtering: {len(y_filtered)}")

# --- Step 1: Encode filtered labels to contiguous integers ---
final_label_encoder = LabelEncoder()
y_filtered_contiguous = final_label_encoder.fit_transform(y_filtered)

num_classes_final = len(final_label_encoder.classes_)

# --- Step 2: Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered,
    y_filtered_contiguous,
    test_size=0.2,
    random_state=42,
    stratify=y_filtered_contiguous
)

print(f"\n[INFO] XGBoost Training Samples: {X_train.shape[0]}, Test Samples: {X_test.shape[0]}")
print(f"[INFO] Number of classes (filtered): {num_classes_final}")

# --- Step 3: Train XGBoost Classifier ---
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # use 'multi:softprob' if you want probabilities
    num_class=num_classes_final,
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

# --- Step 4: Predict and evaluate ---
y_pred = xgb_model.predict(X_test)

used_labels = [int(i) for i in unique_labels(y_test, y_pred)]
target_names = [str(final_label_encoder.classes_[i]) for i in used_labels]

print("\n[CLASSIFICATION REPORT]")
print(classification_report(
    y_test,
    y_pred,
    labels=used_labels,
    target_names=target_names,
    zero_division=0
))


[INFO] Classes before filtering: 44
[INFO] Classes after filtering (>=12 samples): 9
[INFO] Samples after filtering: 5935

[INFO] XGBoost Training Samples: 4748, Test Samples: 1187
[INFO] Number of classes (filtered): 9


/scicomp/home-pure/rqu4/cache/python/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [22:08:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



[CLASSIFICATION REPORT]
              precision    recall  f1-score   support

           3       1.00      1.00      1.00        15
           5       0.92      1.00      0.96        11
          12       1.00      0.82      0.90        11
          13       0.99      1.00      0.99       766
          18       1.00      1.00      1.00       232
          20       1.00      0.65      0.79        23
          21       1.00      1.00      1.00        38
          23       0.98      1.00      0.99        80
          36       1.00      0.91      0.95        11

    accuracy                           0.99      1187
   macro avg       0.99      0.93      0.95      1187
weighted avg       0.99      0.99      0.99      1187

